# 🤖 Aula 5: Criando um Chatbot que Toma Decisões

Neste notebook, vamos construir um chatbot que pode tomar decisões e realizar ações automaticamente, com foco em conceitos técnicos, matemáticos e exemplos de código mais robustos.

## 📌 O que faremos:
1. Criar um chatbot que avalia perguntas e escolhe respostas apropriadas.
2. Integrar chamadas a APIs externas para obter dados em tempo real.
3. Automatizar tarefas como lembretes e notificações.
4. Explorar conceitos matemáticos e técnicos por trás da tomada de decisões.


## 🧠 Conceitos Técnicos e Matemáticos

### Tomada de Decisões em AI Agents
Um AI Agent toma decisões com base em **regras pré-definidas** ou **aprendizado de máquina**. A decisão pode ser modelada como uma função que mapeia entradas (perguntas) para saídas (ações ou respostas):

$$ D: \text{Entrada} \rightarrow \text{Ação} $$

Onde:
- **Entrada**: Pergunta ou comando do usuário.
- **Ação**: Resposta ou tarefa executada pelo chatbot.

### Integração com APIs Externas
APIs externas permitem que o chatbot acesse dados em tempo real, como previsão do tempo, notícias ou informações de banco de dados. A integração é feita através de requisições HTTP, como GET ou POST.

### Automação de Tarefas
A automação envolve a execução de tarefas repetitivas ou programadas, como enviar e-mails, criar lembretes ou atualizar planilhas. Isso pode ser feito usando bibliotecas como `requests`, `smtplib` ou `schedule`.


In [ ]:
# Instale as bibliotecas necessárias
!pip install openai requests schedule

In [ ]:
import openai
import requests
import schedule
import time

# Defina sua chave de API (substitua 'SUA_CHAVE_AQUI' pela chave real)
openai.api_key = "SUA_CHAVE_AQUI"

In [ ]:
# Chatbot que toma decisões baseado na entrada do usuário
def chatbot_decisao(pergunta):
    if "clima" in pergunta.lower():
        return "Quer que eu busque a previsão do tempo para você?"
    elif "notícia" in pergunta.lower():
        return "Posso te mostrar as últimas manchetes de hoje!"
    elif "lembrete" in pergunta.lower():
        return "Qual lembrete você gostaria de criar?"
    else:
        return "Não tenho certeza, mas posso pesquisar isso para você!"

# Teste do chatbot
print(chatbot_decisao("Como está o clima hoje?"))
print(chatbot_decisao("Quais são as notícias mais recentes?"))

In [ ]:
# Função para buscar a previsão do tempo
def buscar_clima(cidade="São Paulo"):
    api_key = "SUA_CHAVE_DE_CLIMA"  # Substitua pela sua chave da API de clima
    url = f"https://api.weatherapi.com/v1/current.json?key={api_key}&q={cidade}"
    resposta = requests.get(url).json()
    return f"A temperatura em {cidade} é {resposta['current']['temp_c']}°C."

# Teste da função
print(buscar_clima("Rio de Janeiro"))

In [ ]:
# Chatbot aprimorado que toma decisões e realiza ações
def chatbot_avancado(pergunta):
    if "clima" in pergunta.lower():
        return buscar_clima()
    elif "notícia" in pergunta.lower():
        return "Aqui estão as notícias mais recentes! (Exemplo)"
    elif "lembrete" in pergunta.lower():
        return "Me diga o que eu devo lembrar e o horário."
    else:
        return "Me diga como posso ajudar!"

# Teste do chatbot
print(chatbot_avancado("Me diga como está o clima em Salvador."))

In [ ]:
import datetime

# Função para criar um lembrete
def criar_lembrete(texto, hora):
    return f"Lembrete criado: {texto} às {hora}!"

# Teste do lembrete
print(criar_lembrete("Reunião de equipe", "14:00"))

## 🛠️ Exemplo Prático: Chatbot com Automação de Tarefas

Vamos criar um chatbot que agenda lembretes e os executa automaticamente.


In [ ]:
def notificar_lembrete(texto):
    print(f"Lembrete: {texto}")

def agendar_lembrete(texto, hora):
    schedule.every().day.at(hora).do(notificar_lembrete, texto)
    print(f"Lembrete agendado: {texto} às {hora}")

# Testando o agendamento
agendar_lembrete("Reunião de equipe", "14:00")

# Mantendo o script rodando para executar lembretes
while True:
    schedule.run_pending()
    time.sleep(1)

## 🎯 Desafios Técnicos

1. **Desafio 1:** Modifique o chatbot para buscar notícias reais usando uma API de notícias (ex.: NewsAPI).
2. **Desafio 2:** Implemente um sistema de **priorização de tarefas** que agenda lembretes com base na urgência.
3. **Desafio 3:** Crie um chatbot que integra múltiplas APIs (ex.: clima, notícias, lembretes) e toma decisões com base no contexto da conversa.

Compartilhe suas soluções no fórum do curso!